In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
import numpy as np

Using TensorFlow backend.


In [0]:

"""
Parameters
"""
epochs = 100
img_width, img_height =300,300
classes_num = 5
batch_size=32
arsitektur = 'Custom1'
#lr = 0.0004

In [0]:
DATA_DIR = '/content/gdrive/My Drive/CITS Color'

train_datagen = ImageDataGenerator(rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    DATA_DIR, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data


Found 1079 images belonging to 5 classes.
Found 268 images belonging to 5 classes.


In [0]:


# (3) Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Convolution2D(filters=8, input_shape=(img_width,img_height,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Convolution2D(filters=16, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Convolution2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Convolution2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Convolution2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(img_width*img_height*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(classes_num))
model.add(Activation('softmax'))

model.summary()


architecure_dir = os.path.join('/content/gdrive/My Drive/Colab Notebooks/LeafClassification/',arsitektur)
if not os.path.exists(architecure_dir):
  os.mkdir(architecure_dir)
# Save the model architecture
with open(os.path.join(architecure_dir,'architecture.json'), 'w') as f:
    f.write(model.to_json())



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 8)         2912      
_________________________________________________________________
activation_1 (Activation)    (None, 73, 73, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 8)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 36, 36, 8)         32        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 16)        15504     
_________________________________________________________________
activation_2 (Activat

In [0]:
from keras.models import model_from_json
with open(os.path.join(architecure_dir,'architecture.json'), 'r') as f:
    model = model_from_json(f.read())

In [0]:
# (4) Compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
"""
Tensorboard log
"""


log_dir = os.path.join(architecure_dir, 'tf-log')
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True)
cbks = [tb_cb]

"""model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=cbks,
    validation_steps=validation_steps)"""

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    callbacks=cbks,
    epochs = epochs,
    shuffle = True)

#model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), steps_per_epoch=len(x_train) / 32, epochs=epochs)

target_dir =os.path.join(architecure_dir, 'models')
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
  
model.save(os.path.join(target_dir, 'model.h5'))
model.save_weights(os.path.join(target_dir, 'weights.h5'))

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
33/33 [==============================] - 500s 15s/step - loss: 0.9683 - acc: 0.7225 - val_loss: 1.0673 - val_acc: 0.7891
Epoch 2/100
33/33 [==============================] - 29s 868ms/step - loss: 0.5382 - acc: 0.8102 - val_loss: 0.7095 - val_acc: 0.8305
Epoch 3/100
33/33 [==============================] - 21s 625ms/step - loss: 0.4194 - acc: 0.8549 - val_loss: 1.1669 - val_acc: 0.7924
Epoch 4/100
33/33 [==============================] - 21s 627ms/step - loss: 0.4488 - acc: 0.8462 - val_loss: 0.9733 - val_acc: 0.8051
Epoch 5/100
33/33 [==============================] - 21s 631ms/step - loss: 0.4031 - acc: 0.8468 - val_loss: 0.5092 - val_acc: 0.8347
Epoch 6/100
33/33 [==============================] - 21s 636ms/step - loss: 0.3806 - acc: 0.8631 - val_loss: 0.2805 - val_acc: 0.8856
Epoch 7/100
33/33 [==============================] - 21s 622ms/step - loss: 0.3844 - acc: 0.8508 - val_loss: 0.2742 - val_acc: 0.9068
Epoch 8/100
33/